In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo",]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

### [1. Forward Imaging Model](fwdimaging.ipynb)

## 2. The Recovery Process
This process recover the Hi-Res complex object using the Lo-Res image set. Each image of it is generated by the lighting that emerges from a different LED of a grid: the forward imaging process ([fwdimaging.ipynb](fwdimaging.ipynb)).

In [ ]:
import fwdimaging as fwd
import numpy as np
import scipy.misc
import matplotlib.pyplot as plt
import os

In [ ]:
# Return a list with the indexes of the images for the reconstruction
# sequence or -1 ('int') if the size of the LED grid is not odd in
# which case there is no an unique LED as center.
# The first index is the center. The way is spiral-shaped.
def get_idx_seq_img(grid_len, **kwargs):
    clockwise = kwargs.pop('clockwise', True)
    if grid_len % 2 != 0:
        l_idx_seq_img = list()
        img = int(((grid_len ** 2) - 1) / 2)  #Start in the center.
        l_idx_seq_img.append(img)
        l_addends = [(grid_len, -1), (-grid_len, 1)]
        if not clockwise:
            l_addends = [(grid_len, 1), (-grid_len, -1)]
        # Index to choose between (grid_len, -1) ('tuple') and
        # (-grid_len, 1).
        i = 0
        c = 1
        last_img = (grid_len ** 2) - 1
        while True:
            i %= 2
            v_addend, h_addend = l_addends[i]
            for j in range(c):
                img += v_addend
                if img > last_img:
                    break
                else:
                    l_idx_seq_img.append(img)
            for j in range(c):
                img += h_addend
                if img > last_img:
                    break
                else:
                    l_idx_seq_img.append(img)
            if img >= last_img:
                return l_idx_seq_img
            else:
                i += 1
                c += 1
    else:
        return -1

In [ ]:
# Return the Hi-Res object recovered by applyin Fourier
# Ptychography algorithm.
def get_photo(**kwargs):
    inpath = kwargs.pop('inpath', 'img-lores/')
    inprefix = kwargs.pop('inprefix', 'lores_')
    inextension = kwargs.pop('inextension', '.tiff')
    firstsuffix = kwargs.pop('firstsuffix', '001')
    outpath = kwargs.pop('outpath', 'out-img-hires/')
    # rows ('int') is the number of LEDs per row and column. It could
    # be simple remember the question "how many rows of LEDs does my
    # grid have?". The array will be rows-by-rows order.
    # For rows ('int'), 8 is the default value by the Adafruit
    # NeoPixel NeoMatrix 8x8 (https://www.adafruit.com/product/1487).
    # The default values of dist ('int') and h ('int') is based on the
    # use of that grid.
    rows = kwargs.pop('rows', 15)
    # rows = kwargs.pop('rows', 8)
    # The length of the centered row with the LEDs to use. It is also
    # used for the range of rows.
    rowsuse = kwargs.pop('rowsuse', 15)
    dist = kwargs.pop('dist', 4)
    # dist = kwargs.pop('dist', 9)  # Distance in mm between LEDs.
    # Distance in mm between the LED grid and the sample.
    height = kwargs.pop('height', 90)  
    # height = kwargs.pop('height', 100)
    ###
    # Simulate the set of Lo-Res images.
    sim = kwargs.pop('sim', False)
    amplitude = kwargs.pop('amplitude', 'img/cameraman.tif')
    phase = kwargs.pop('phase', 'img/westconcordorthophoto.png')
    ###
    # Parameters of the coherent imaging system.
    wavelen = kwargs.pop('wavelen', 0.63e-6)
    # Sampling pixel size of the CCD.
    ccdpx = kwargs.pop('ccdpx', 2.75e-6)
    # Pixel size of the reconstruction.
    hirespx = kwargs.pop('hirespx', ccdpx / 4)
    na = kwargs.pop('na', 0.08)  # Numerical aperture of the employed
                                 # objective lens.
    # DANGER! IT COULD PIXELATE the output image.
    zoom = kwargs.pop('zoom', 4)
    ###
    loops = kwargs.pop('loops', 5)
    # Show the output Hi-Res image in the notebook.
    name = kwargs.pop('name', 'out-img-hires')  # Name of the file.
    extension = kwargs.pop('extension', '.tiff')
    show = kwargs.pop('show', True)
    # The wavevectors.
    wvs = fwd.generate_wave_vectors(rows, dist, height)
    if not sim:  # Import your own Lo-Res image set.
        return -1
        ''' subsel = fwd.get_idx_img_to_use(rows, rowsuse, matlab=True)
        img_seq_lores = fwd.get_set_from_folder(subsel, path=inpath,
                                                 prefix=inprefix,
                                                 extension=inextension)
        hpx, wpx = img_seq_lores[0].shape
        hpx *= zoom  # Height of the output Hi-Res image.
        wpx *= zoom  # ... the width.
        p, q, dkx, dky, kx, ky, cft = fwd.get_cft(wvs, wavelen, ccdpx,
                                                  zoom, na, hpx, wpx)
        '''
    else:
        in_obj, hpx, wpx = fwd.generate_obj(amplitude, phase)
        p, q, dkx, dky, kx, ky, cft = fwd.get_cft(wvs, wavelen, ccdpx,
                                                  zoom, na, hpx, wpx)
        img_seq_lores = fwd.generate_lores_set(in_obj, cft, rows, kx,
                                               ky, hpx, wpx, dkx, dky,
                                               p, q)
    seq = get_idx_seq_img(rows)
    obj_recover = np.ones((hpx, wpx), dtype=np.int)
    obj_recover_ft = np.fft.fftshift(np.fft.fft2(obj_recover))
    for i in range(loops):
        for j in range(rows ** 2):
            k = seq[j]
            kxc = fwd.round_half_up((wpx + 1) / 2.0 + kx[k] / dkx)
            kyc = fwd.round_half_up((hpx + 1) / 2.0 + ky[k] / dky)
            kyl = fwd.round_half_up(kyc - (p - 1) / 2.0)
            kyh = fwd.round_half_up(kyc + (p - 1) / 2.0)
            kxl = fwd.round_half_up(kxc - (q - 1) / 2.0)
            kxh = fwd.round_half_up(kxc + (q - 1) / 2.0)
            lores_ft = ((p / hpx) ** 2) * obj_recover_ft[kyl - 1:kyh, kxl - 1:kxh]
            lores_ft *= cft
            img_lores = np.fft.ifft2(np.fft.ifftshift(lores_ft))
            img_lores = ((hpx / p) ** 2) * img_seq_lores[k] * np.exp(1j * np.angle(img_lores))
            lores_ft = np.fft.fftshift(np.fft.fft2(img_lores)) * cft
            obj_recover_ft[kyl - 1:kyh, kxl - 1:kxh] = (1 - cft) * obj_recover_ft[kyl - 1:kyh, kxl - 1:kxh] + lores_ft
    obj_recover = np.fft.ifft2(np.fft.ifftshift(obj_recover_ft))
    obj_recover = np.absolute(obj_recover)
    if not os.path.exists(outpath):
        os.makedirs(outpath)
    scipy.misc.imsave(outpath + name + extension, obj_recover)
    if show:
        plt_img = plt.imshow(obj_recover, cmap='Greys_r')
    return obj_recover